# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

#uv_install("botocore", "boto3")


In [ ]:
import os
import agent_core_utils

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' # set the AWS region

In [ ]:
#### Create an IAM role for the Gateway to assume
import agent_core_utils
agentcore_gateway_iam_role = agent_core_utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

In [ ]:


# Creating Cognito User Pool
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = agent_core_utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

agent_core_utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = agent_core_utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)



In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='LambdaOrderGateway',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

In [ ]:
# Replace the AWS Lambda function ARN below
order_by_id_lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": '${YourGetOrderByIdLambdaFunctionARN}', # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_by_id_tool",
                        "description": "tool to get the order by id",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "id": {
                                    "type": "string"
                                }
                            },
                            "required": ["id"]
                        }
                    }
                ]
            }
        }
    }
}

order_by_created_dates_lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": '${YourGetOrdersByCreatedDatesLambdaFunctionARN}', # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_orders_by_created_dates_tool",
                        "description": "tool to get the order by created dates",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "startDate": {
                                    "type": "string"
                                },

                                "endDate": {
                                    "type": "string"
                                }
                            },
                            "required": ["startDate","endDate"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
order_by_id_targetname='GetOrderByIdLambda'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=order_by_id_targetname,
    description='Order by id Lambda Target',
    targetConfiguration=order_by_id_lambda_target_config,
    credentialProviderConfigurations=credential_config)

order_by_id_targetname='GetOrderByCreatedDatesLambda'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=order_by_id_targetname,
    description='Orders by created dates Lambda Target',
    targetConfiguration=order_by_created_dates_lambda_target_config,
    credentialProviderConfigurations=credential_config)

In [ ]:
import agent_core_utils

print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = agent_core_utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

In [ ]:
from strands import Agent
import logging
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import os
import requests
import json


def fetch_access_token():
    return token;


def create_streamable_http_transport(mcp_url: str, access_token: str):
    return streamablehttp_client(
        mcp_url,
        #timeout=1,
        headers={"Authorization": f"Bearer {token}"})


def get_full_tools_list(client):
    """
    List tools w/ support for pagination
    """
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools


def run_agent(mcp_url: str, access_token: str):
    model = BedrockModel(
        model_id="us.amazon.nova-pro-v1:0",
        temperature=0.7)
    mcp_client = MCPClient(lambda: create_streamable_http_transport(mcp_url, access_token))

    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print(f"Found the following tools: {[tool.tool_name for tool in tools]}")

        agent = Agent(model=model, tools=tools)
        #agent("Give me the information about order with id 12345 ")
        #agent('In case you found the order, can you provide the information about the user who made #this purchase')
        agent('Can you list orders created between 1 of August 2025 5am and 5 of August 2025 3am. '
              'Please use the following date format, for example: 2025-08-02T19:50:55')
        agent('What is the total value of all these orders together?')


gateway_url = "${YOUR_Gateway_resource_URL}"
run_agent(gateway_url, fetch_access_token())